In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
url = '/Users/pierre/code/PierreCeraH/deepagri/raw_data/cleaned/Histo_data_YieldsAreasProd.xlsx'
df = pd.read_excel(url, sheet_name='Blé tendre')

In [3]:
def get_df_all(df):
    df = df.rename(columns={'Unnamed: 0':'Année'})
    df = df.set_index('Année')

    col_rendements = []
    col_surfaces = []
    col_prod = []

    for col in df.columns :
        if '1' in col: #rendements
            col_rendements.append(col)
        elif '2' in col: #production
            col_prod.append(col)
        else: #surfaces
            col_surfaces.append(col)
            
    df_rendements = df[col_rendements].drop(['Année',2022],axis=0)
    df_prod = df[col_prod].drop(['Année',2022],axis=0)
    df_surfaces = df[col_surfaces].drop(['Année',2022],axis=0)     
    
    dept_codes = [1,2,3,4,5,6,7,8,9,10,
              11,12,13,14,15,16,17,
              18,19,
              21,22,23,24,25,26,27,28,29,
              20,
              30,31,32,33,34,35,36,
              37,38,39,40,41,42,43,
              44,45,46,47,48,49,50,
              51,52,53,54,55,56,57,
              58,59,60,61,62,63,64,
              65,66,67,68,69,70,71,
              72,73,74,75,76,77,78,
              79,80,81,82,83,84,85,
              86,87,88,89,90,91,92,
              93,94,95,96,0]
    
    col_names_rend = df_rendements.columns
    col_names_surf = df_surfaces.columns
    col_names_prod = df_prod.columns
    
    dict_rename_rend = {}
    dict_rename_surf = {}
    dict_rename_prod = {}
    
    i = 0
    for col in col_names_rend:
        dict_rename_rend[col]=dept_codes[i]
        i = i+1
    
    i = 0
    for col in col_names_surf:
        dict_rename_surf[col]=dept_codes[i]
        i = i+1
    
    i = 0
    for col in col_names_prod:
        dict_rename_prod[col]=dept_codes[i]
        i = i+1
        
    df_rendements = df_rendements.rename(columns=dict_rename_rend)
    df_rendements = df_rendements.rename(columns={96:'FRANCE'})
    
    df_prod = df_prod.rename(columns=dict_rename_prod)
    df_prod = df_prod.rename(columns={96:'FRANCE'})
    
    df_surfaces = df_surfaces.rename(columns=dict_rename_surf)
    df_surfaces = df_surfaces.rename(columns={96:'FRANCE'})
    
    return df_rendements, df_surfaces, df_prod

In [4]:
def baseline(df_rendements, df_surfaces):
    baseline_rendements = {}
    for col in df_rendements.columns:
        baseline_rendements[col] = df_rendements[col].mean()
    
    baseline_surfaces = {}
    for col in df_surfaces.columns:
        baseline_surfaces[col] = df_surfaces[col].mean()

    base_rendements = pd.DataFrame.from_dict(baseline_rendements,orient='index')
    base_surfaces = pd.DataFrame.from_dict(baseline_surfaces,orient='index')
    
    base_rendements = base_rendements.rename(columns={0:'Rendements'})
    base_surfaces = base_surfaces.rename(columns={0:'Surfaces'})
    
    baseline = base_rendements.copy()
    baseline = baseline.join(base_surfaces)
    
    baseline['Production']=baseline['Rendements']*baseline['Surfaces']/10
    
    return baseline

In [5]:
df_rendements, df_surfaces, df_prod = get_df_all(df)
baseline = baseline(df_rendements, df_surfaces)

In [6]:
baseline.to_csv('baseline.csv')